In [1]:
import cv2
import sys
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import imutils #thru the pip package.
# from skimage.filters import threshold_adaptive
import gtk, pygtk


In [2]:
# Some globals -
window = gtk.Window()
screen = window.get_screen()
# for positioning image windows
windowX,windowY = 0,0 
windowWidth = screen.get_width()
windowHeight = screen.get_height()


In [3]:
# Toolbox functions-
# cv2.createTrackbar('boxDim', 'ImageWindow', 2000, 5000, someFunctionCallBack)

def waitQ():
    while(cv2.waitKey(1)& 0xFF != ord('q')):pass
    cv2.destroyAllWindows()

def show(name,img,pause=1,resetpos=None):
    global windowX
    global windowY
    if(type(img)== type(None)):
        print(name," NoneType image to show!")
        if(pause):
#             debug = input()
            cv2.destroyAllWindows()
        return
    cv2.imshow(name,img)
    
    w,h,_ = img.shape
    if(resetpos):
        windowX=resetpos[0]
        windowY=resetpos[1]
    cv2.moveWindow(name,windowX,windowY)
    overflowX = windowX+w > windowWidth
    overflowY = windowY+h > windowHeight
    if(overflowX):
        windowX = 0
        if(not overflowY):windowY+=h
    else:windowX+=w
    if(overflowY):windowY = 0
    if(pause):
        waitQ()
    
def myColor1():
    return (randint(100,250),randint(100,250),randint(100,250))
def order_points(pts):
	rect = np.zeros((4, 2), dtype = "float32")

	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	# compute the width of the new image, which will be the
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	# compute the height of the new image, which will be the
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	# return the warped image
	return warped
    

def readResponse(pts,boxDim,img,threshold=0.5,bord=-1,white=(220,220,220),black=(0,0,0)):
    _, t = cv2.threshold(img,100,255,cv2.THRESH_BINARY)
    w,h=boxDim
    mask = 255*np.ones(boxDim, np.uint8)
    ret = {}
    for pt in pts:
        pt = tuple(map(int,pt))
        x,y=pt 
        crop=img[y:y+h,x:x+w]
        mean_color = cv2.mean(crop,mask)
        ret[pt]= mean_color[0]/255#
        clr = black if threshold > mean_color[0]/255 else white
        cv2.rectangle(img,pt,(x+w,y+h),clr,bord)#-1 is for fill
    return ret
def calcGaps(PointsX,PointsY,numsX,numsY):
    gapsX = ( abs(PointsX[0]-PointsX[1])/(numsX[0]-1),abs(PointsX[2]-PointsX[3]) )
    gapsY = ( abs(PointsY[0]-PointsY[1])/(numsY[0]-1),abs(PointsY[2]-PointsY[3]) )
    return (gapsX,gapsY)

def maketemplate(start,numsX,numsY,gapsX,gapsY):
    templateMCQ=[]
    posx=start[0]
    for x in range(numsX[1]):
        posy=start[1]
        for y in range(numsY[1]):
            templateMCQ.append((posx,posy))
            posy+= (gapsY[1] if ((y+1) % numsY[0]==0) else gapsY[0])
        posx+= (gapsX[1] if ((x+1) % numsX[0]==0) else gapsX[0])
    return templateMCQ



In [4]:
# Good ones from opencv SAMPLES -
"""
mouse_and_match - template matching
lk_track - draw in the air
lk_homography - see good features
gabor_threads - that fantasy tiger wallpaper
floodfill -> That paint bucket is this one
"""


'\nmouse_and_match - template matching\nlk_track - draw in the air\nlk_homography - see good features\ngabor_threads - that fantasy tiger wallpaper\nfloodfill -> That paint bucket is this one\n'

In [5]:
a=[0.5,1.5,0,1,3,5,6]
map(int,np.multiply(a,10))
a = [a,a]
pts = [[0,1],[1,0],[1,5],[0,5]]
map(lambda x:a[x[0]][x[1]],sorted(pts,key=lambda pt: a[pt[0]][pt[1]],reverse=True))

[5, 5, 1.5, 0.5]

In [6]:
def dist(p1,p2):
    return np.linalg.norm(np.array(p1)-np.array(p2))
def getslope(pt1,pt2):
    return float(pt2[1]-pt1[1])/float(pt2[0]-pt1[0])
def check_min_dist(pt,pts,min_dist):
    for p in pts:
        if(dist(pt,p) < min_dist):
            return False
    return True
        

def get_reflection(pt, pt1,pt2):
    pt, pt1,pt2 = map(lambda x:np.array(x,dtype=float),[pt, pt1,pt2])
    return (pt1 + pt2) - pt
def printbuf(x):
    sys.stdout.write(str(x))
    sys.stdout.write('\r')

def someFunctionCallBack(*arg):
    readResponse(pts,(boxDimX,boxDimY),img,threshold=0.55,bord=-1)

def match_template_scaled(img1, template,show_detected=False,pts=4,
                          scaleRange=(0.5,1.5),fac=100,min_dist=None,threshold=0.6,
                          CLR=(0,255,255),iterLim=30,excludepts=None):
    orig=template.copy()
    w1,h1,_=img1.shape
    w,h,_=template.shape
    if( not min_dist):
        min_dist = h1/10
    scale=float(h1)/h
    print('initial scale',scale)
    x=map(int,np.multiply(scaleRange,fac))
    if((x[1]-x[0])> iterLim*fac/10):
        print("Too many iterations : %d, reduce scaleRange" % ((x[1]-x[0])*10/fac) )
        return []
    r_max=None
    for r0 in range(x[0],x[1],fac/10):
        r=float(r0)/fac
        printbuf(r)
        if(r==0.0):
            continue
        templ = imutils.resize(orig, height = int(h*r))
        res = cv2.matchTemplate(img1,templ,cv2.TM_CCOEFF_NORMED)
#         cv2.imshow('res',res)        waitQ()
        maxT = res.max()
        if(threshold < maxT):
            printbuf(' %d )Better match %d%%' % (r0,100*maxT))
            r_max=r
            threshold = maxT
    if(r_max==None):
        print("No matchings for given threshold & scaleRange")
        return []
    print('')
    print('final scale',r_max)
    templ = imutils.resize(orig, height = int(h*r_max))
    res = cv2.matchTemplate(img1,templ,cv2.TM_CCOEFF_NORMED)# better than TM_CCOEFF_NORMED)
#     show('res',res)    
    #Set threshold of top four points - 
    #argpartition : kth point will be in its sorted position i.e. k, others will be 'partitioned'
    res2  =  res.flatten() # serial threshold values.
    partitioned_res2=np.argpartition(a = res2,kth = -pts*2)
    threshold = min(res2[partitioned_res2[-pts*2:]])

#     NOW DISCARD NEARBY POINTS
    loc = np.where(res>=threshold) # now this already gives only pts*2 points
    locs = zip(*loc[::-1])
#     Now just order them by res values
    locs = sorted(locs,key=lambda pt: res[pt[1]][pt[0]],reverse=True)
    w,h,_=templ.shape
    centres=[]
    i=0

    exclude = type(excludepts)!=type(None)
    
    for pt in locs: #zipped because it gives Xs and Ys seperated & probab ordered by Xs
        if(i==pts):
            break
        if(check_min_dist(pt,centres,min_dist) or (exclude and check_min_dist(pt,excludepts,min_dist))):
            cv2.rectangle(img1,pt,(pt[0]+w,pt[1]+h),CLR,2)
            centres.append([pt[0]+w/2,pt[1]+h/2])
            i+=1
        else:
            print('skipped nearby point', pt,min_dist,centres,excludepts)
        
    if(show_detected):
        show('detected',img1)    
    return np.array(centres)

def get_fourth_pt(three_pts):
    m=[]
    # 	of the three points, two form a diagonal, third one can be found by-
#         1. Angle estimate : In the triangle, choose the one having near 90 degrees and add its reflection
#         2. Slope estimation-
#         3. Distance calc

    for i in range(3):
        m.append(dist(three_pts[i],three_pts[(i+1)%3]))
        
    v =max(m)
    for i in range(3):
        if(m[i]!=v and m[(i+1)%3]!=v):
            refl = (i+1) % 3
            break
    fourth_pt = get_reflection( three_pts[refl],three_pts[(refl+1)%3],three_pts[(refl+2)%3])
    return fourth_pt

In [7]:
# The Wrapper
def getROI(orig,templ,excludetempl=None,verbose=False,pts=3,scaleRange=(0.25,1.5),fac=100,threshold=0.55,iterLim=50):
    template=templ.copy()
    image=orig.copy()
    image=image-cv2.erode(image,None)
    template=template-cv2.erode(template,None)
    excludepts=None
    if( type(excludetempl) != type(None)):
        excludetempl=excludetempl-cv2.erode(excludetempl,None)
        excludepts = match_template_scaled(image,excludetempl,
                                show_detected=verbose,
                                pts=1,
                                scaleRange=scaleRange,fac=fac,threshold=threshold,iterLim=iterLim)
        print("lon location : ",excludepts)
    
        
    
    # cv2.createTrackbar('boxDim', 'ImageWindow', 2000, 5000, match_template_scaled_with_time_flag)
    four_pts =match_template_scaled(image,template,
                                show_detected=verbose,
                                pts=pts,
                                scaleRange=scaleRange,fac=fac,threshold=threshold,iterLim=iterLim,excludepts=excludepts)
    if(pts == 3 and len(four_pts)==3):
        three_pts=four_pts
        four_pts = np.concatenate([four_pts,[get_fourth_pt(three_pts)]])

    if(verbose):
        print(four_pts)
    if(len(four_pts)>=4):
        warped = four_point_transform(orig, four_pts)
        return warped        
    else:
        print("Unable to find enough points!")
        return None

In [8]:
# How about cropping out top 5% and bottom 25% part of the input?

windowX,windowY = 0,0 
uniform_height = 1000
template = cv2.imread('FinalCircle.png') #,cv2.CV_8UC1/IMREAD_COLOR/UNCHANGED
lontemplate = cv2.imread('lontemplate.jpg') 
# orig = cv2.imread('practiOMR.jpg') #,cv2.CV_8UC1/IMREAD_COLOR/UNCHANGED
# orig2 = cv2.imread('practiOMR2.jpg') #,cv2.CV_8UC1/IMREAD_COLOR/UNCHANGED
# orig = imutils.resize(orig,height=uniform_height)
# orig2 = imutils.resize(orig2,height=uniform_height)

# warped = getROI(orig,template,scaleRange=(0.75,1.5))
# show('cropped',warped,0)
# warped2 = getROI(orig2,template,scaleRange=(0.75,1.5))
# show('cropped2',warped2,0)

for i in range(1,36):
    sample3 = cv2.imread('realsamples/sample'+str(i)) 
    sample3 = imutils.resize(sample3,height=uniform_height)
    sample3crop = getROI(sample3,template,scaleRange=(0.75,1.5))
    show('sample'+str(i),sample3crop)
    

#Getting Wrong Crops? - Keep checking with the scales

# warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)`

In [9]:
"""
Now dvlpments-
If multi-marked, try lower threshold, if still get multiple dark, move on, else choose the most dark one.
Make a questions class - It has options & their (4 by default) coordinates
pass array of questions to readResponse : it just reads the coords and updates whether its marked or not.

"""
drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1
fx,fy = -1,-1
erase= True

# mouse callback function
def draw_rect(event,x,y,flags,param):
    global ix,iy,fx,fy,drawing,mode,mask

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.rectangle(mask,(ix,iy),(x,y),(0,0,0),-1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if(erase):
            mask = np.zeros_like(mask)
        print("(%d,%d) : (%d,%d)" % (ix,iy,x,y))
        cv2.putText(mask,("(%d,%d) : (%d,%d)" % (ix,iy,x,y)),(10,30),cv2.FONT_HERSHEY_SIMPLEX,
                    1.0,(0,250,100),2)
        fx,fy=x,y
        cv2.rectangle(mask,(ix,iy),(fx,fy),(0,0,0),-1)
# image was grayscale
img = warped#np.zeros(warped.shape, np.uint8)
mask = np.zeros(img.shape, np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_rect)
pts1=[]
pts2=[]
while(1):
    cv2.imshow('image',cv2.subtract(img,mask))
    
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break
    elif k == ord('x'):
        print("Erase",erase)
        erase=not erase
        
    elif k == ord('a'):
        pt1,pt2=(ix,iy),(fx,fy)        
        print("added",pt1,pt2)
        pts1.append(pt1)
        pts2.append(pt2)
        
cv2.destroyAllWindows()

pt1,pt2=np.array(pts1).mean(),np.array(pts2).mean()

NameError: name 'warped' is not defined

In [ ]:
def mktempl(pX,pY,divsX,divsY):
    gapsX,gapsY=calcGaps(pX,pY,nX,nY)
    return maketemplate((pX[0],pY[0]),nX,nY,gapsX,gapsY)

img = cv2.imread('technoOMR_marked.jpg',cv2.CV_8UC1) #IMREAD_COLOR/UNCHANGED
h,w =img.shape
PointsX,PointsY = (38,92,92,118.75),(352,388,388,406.25)
gapsX,gapsY=calcGaps(PointsX,PointsY,(4,20),(5,20))
print(gapsX,gapsY)
# PointsX will have only x coords of the horiz points
t=mktempl(PointsX,PointsY,divsX=(4,20),divsY=(5,20))
g1,g2=calcGaps((265,282,282,310),(236,317,236,317),(2,4),(10,10))
t1=maketemplate((265,236),(2,4),(10,10),g1,g2)
t2=maketemplate((355,236),(2,4),(10,10),(g1[0]*1.13,g1[0]),g2)
t3=maketemplate((355,236),(2,4),(10,10),(g1[0]*1.13,g1[0]),g2)
# t3=maketemplate((30,90),(25,25),(26,26),(9,9),PointsY)
pts=t+t1+t2
boxDimX,boxDimY=(6,6)
readResponse(pts,(boxDimX,boxDimY),img,threshold=0.55,bord=-1)
cv2.imshow('imageWindow',img)
# cv2.createTrackbar('boxDimY', 'imageWindow', 6, 10, someFunctionCallBack)

waitQ()
"""
Note : You'll get this error if the code is accessing pixels out of bounds of the image
error: (-215) A.size == arrays[i0]->size in function init
"""

In [ ]:
"""
Pseudo code- Sound and Simple
Image processing to :
    Done using getPerspectiveTransform> Scale and rotate image correctly
    Detect responses correctly-
        Check if bubble size bigger than threshold
    Return the response to each question in a list.
Pure python to :
    Check the answers
    Provide a GUI ? (tkinker)
    >Web interface preferred
    
    
But first, get going with a static template and read the color
"""

In [ ]:
img = cv2.imread('OMRMarked.jpg',cv2.CV_8UC1) #IMREAD_COLOR/UNCHANGED
 
#---- 4 corner points of the bounding box
pts_src = np.array([[17.0,0.0], [77.0,5.0], [0.0, 552.0],[53.0, 552.0]])

#---- 4 corner points of the black image you want to impose it on
pts_dst = np.array([[0.0,0.0],[77.0, 0.0],[ 0.0,552.0],[77.0, 552.0]])

#---- forming the black image of specific size
im_dst = np.zeros((552, 77, 3), np.uint8)

#---- Framing the homography matrix
h, status = cv2.findHomography(pts_src, pts_dst)

#---- transforming the image bound in the rectangle to straighten
im_out = cv2.warpPerspective(img, h, (im_dst.shape[1],im_dst.shape[0]))
cv2.imshow("im_out.jpg", im_out)
waitQ()

In [ ]:
#underscore consumes the variable - retval here
img = cv2.imread('OMRSample.jpg',cv2.CV_8UC1) #IMREAD_COLOR/UNCHANGED
#retval, threshold = cv2.threshold(img,signumThrVal,maxVal,cv2.THRESH_BINARY_INV)
threshold=[]
_, t = cv2.threshold(img,100,255,cv2.THRESH_BINARY)
threshold.append(t)
threshold.append(cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,115,1))
ret, t = cv2.threshold(img,126,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
threshold.append(t)
cv2.imshow('imageWindow',img)

for i in range(0,3):
    cv2.imshow('th'+str(i),threshold[i]);
    cv2.moveWindow('th'+str(i),1280,0)

waitQ()
    

In [ ]:
"""Cam Scanner Code 
image = cv2.imread('OMRTemp.jpg') #,cv2.CV_8UC1/IMREAD_COLOR/UNCHANGED
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
# print "STEP 1: Edge Detection"
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
waitQ()

# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
# try mode= CV_RETR_EXTERNAL 
_ , cnts ,_ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#The hack -
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:min(5,len(cnts))] # get top 5 largest contours by area

# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)

	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break

# show the contour (outline) of the piece of paper
print "STEP 2: Find contours of paper"
cv2.drawContours(image, [screenCnt], -1, (244, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)

# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
# warped = threshold_adaptive(warped, 251, offset = 10)
# warped = cv2.adaptiveThreshold(warped,25,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,115,1)
warped = warped.astype("uint8") * 255
 
# show the original and scanned images
print "STEP 3: Apply perspective transform"
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
waitQ()


In [ ]:
"""
Template Matching Code-
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = cv2.imread('technoOMR_marked.jpg',0)
img2 = img.copy()
template = cv2.imread('circle.jpg',0)
w, h = template.shape[::-1]

# All the 6 methods for comparison in a list
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
for meth in methods:
    img = img2.copy()
    method = eval(meth)
    # Apply template Matching
    res = cv2.matchTemplate(img,template,method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img,top_left, bottom_right, 0, 2)
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img,cmap = 'gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.suptitle(meth)
    plt.show()
"""